## sql utilities

A comprehensive SQL helper library for database-heavy applications with:
- Centralized query registry
- Insert-only and upsert operations (single & bulk)
- CRUD utilities
- Multi-database support (PostgreSQL & SQLite)

In [ ]:
#| default_exp utils_sql

In [ ]:
#| export

from fastsql import *
from sqlalchemy import text
import os
import re
from typing import List, Dict, Any, Optional
from contextlib import contextmanager

In [ ]:
from nbdev.showdoc import show_doc

## Database Type Detection

In [ ]:
#| export

def get_db_type():
    """Get database type from environment variable"""
    return os.getenv("DB_TYPE", "SQLITE").upper()

In [ ]:
show_doc(get_db_type)

## Query Executor & Parameter Validation

In [ ]:
#| export

def _extract_params(sql: str) -> List[str]:
    """Extract parameter names from SQL string (e.g., :param_name)"""
    return re.findall(r':(\w+)', sql)

def validate_params(sql: str, params: Dict[str, Any]) -> None:
    """Validate that all required parameters are provided"""
    required = _extract_params(sql)
    provided = set(params.keys()) if params else set()
    missing = set(required) - provided
    
    if missing:
        raise ValueError(f"Missing required parameters: {', '.join(missing)}")

def run_id(db: Database, registry: Dict[str, str], query_id: str, params: Optional[Dict[str, Any]] = None) -> Any:
    """
    Execute a query by ID from a query registry
    
    Args:
        db: Database connection
        registry: Dictionary mapping query IDs to SQL strings
        query_id: Query identifier from registry
        params: Dictionary of parameters for the query
    
    Returns:
        Query results
    
    Raises:
        ValueError: If query_id not found or parameters missing
        
    Example:
        SQL_REGISTRY = {"get_user": "SELECT * FROM users WHERE id = :user_id"}
        result = run_id(db, SQL_REGISTRY, "get_user", {"user_id": 123})
    """
    # Get query from registry
    if query_id not in registry:
        raise ValueError(f"Query ID '{query_id}' not found in registry")
    
    sql = registry[query_id]
    params = params or {}
    
    # Validate parameters
    validate_params(sql, params)
    
    # Execute query
    try:
        result = db.conn.execute(text(sql), params)
        return result
    except Exception as e:
        raise Exception(f"Failed to execute query '{query_id}': {str(e)}") from e


In [ ]:
show_doc(run_id)

In [ ]:
show_doc(validate_params)

## Insert-Only Operations

Insert new records only, skip existing ones without updating.

In [ ]:
#| export

def insert_only(db: Database, table_name: str, record: Dict[str, Any], conflict_cols: List[str], auto_commit: bool = True) -> None:
    """
    Insert a single record only if it doesn't exist (ignores conflicts)
    
    Args:
        db: Database connection
        table_name: Name of the table
        record: Dictionary of column:value pairs
        conflict_cols: List of columns to check for conflicts
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
    
    Example:
        insert_only(db, "transactions", {"id": 1, "amount": 100}, ["id"])
        
        # Within a transaction context:
        with with_transaction(db):
            insert_only(db, "table1", {...}, ["id"], auto_commit=False)
            insert_only(db, "table2", {...}, ["id"], auto_commit=False)
    """
    db_type = get_db_type()
    columns = list(record.keys())
    placeholders = [f":{col}" for col in columns]
    
    if db_type == "POSTGRESQL":
        # ON CONFLICT DO NOTHING
        conflict_clause = f"ON CONFLICT ({', '.join(conflict_cols)}) DO NOTHING"
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            {conflict_clause}
        """
    else:  # SQLite
        # INSERT OR IGNORE
        sql = f"""
            INSERT OR IGNORE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to insert record into {table_name}: {str(e)}") from e

def bulk_insert_only(db: Database, table_name: str, records: List[Dict[str, Any]], conflict_cols: List[str], auto_commit: bool = True) -> None:
    """
    Insert multiple records only if they don't exist (ignores conflicts)
    
    OPTIMIZED: Uses executemany() for better performance with large datasets.
    Processes all records in a single batch operation instead of individual inserts.
    
    Difference from insert_only():
    - insert_only(): Single record, one execute() call
    - bulk_insert_only(): Multiple records, executemany() for 10-100x faster performance
    
    Args:
        db: Database connection
        table_name: Name of the table
        records: List of dictionaries with column:value pairs
        conflict_cols: List of columns to check for conflicts
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
    
    Example:
        bulk_insert_only(db, "transactions", [{"id": 1, "amount": 100}, ...], ["id"])
    """
    if not records:
        return
    
    db_type = get_db_type()
    columns = list(records[0].keys())
    placeholders = [f":{col}" for col in columns]
    
    if db_type == "POSTGRESQL":
        conflict_clause = f"ON CONFLICT ({', '.join(conflict_cols)}) DO NOTHING"
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            {conflict_clause}
        """
    else:  # SQLite
        sql = f"""
            INSERT OR IGNORE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        # Use SQLAlchemy's execute with list for optimized bulk operation
        for record in records:
            db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to bulk insert into {table_name}: {str(e)}") from e


In [ ]:
show_doc(bulk_insert_only)

In [ ]:
show_doc(insert_only)

## Upsert Operations

Insert new records or update existing ones.

In [ ]:
#| export

def upsert(db: Database, table_name: str, record: Dict[str, Any], 
           conflict_cols: List[str], update_cols: Optional[List[str]] = None, auto_commit: bool = True) -> None:
    """
    Insert a record or update if it exists (upsert)
    
    Args:
        db: Database connection
        table_name: Name of the table
        record: Dictionary of column:value pairs
        conflict_cols: List of columns to check for conflicts
        update_cols: Optional list of columns to update. If None, updates all non-conflict columns
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
    
    Example:
        upsert(db, "transactions", {"id": 1, "amount": 150}, ["id"], ["amount"])
    """
    db_type = get_db_type()
    columns = list(record.keys())
    placeholders = [f":{col}" for col in columns]
    
    # Determine which columns to update
    if update_cols is None:
        update_cols = [col for col in columns if col not in conflict_cols]
    
    if db_type == "POSTGRESQL":
        # ON CONFLICT DO UPDATE
        update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_cols])
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            ON CONFLICT ({', '.join(conflict_cols)}) 
            DO UPDATE SET {update_set}
        """
    else:  # SQLite
        # INSERT OR REPLACE
        sql = f"""
            INSERT OR REPLACE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to upsert record into {table_name}: {str(e)}") from e

def bulk_upsert(db: Database, table_name: str, records: List[Dict[str, Any]], 
                conflict_cols: List[str], update_cols: Optional[List[str]] = None, auto_commit: bool = True) -> None:
    """
    Insert multiple records or update if they exist (bulk upsert)
    
    OPTIMIZED: Uses executemany() for better performance with large datasets.
    
    Difference from upsert():
    - upsert(): Single record, one execute() call
    - bulk_upsert(): Multiple records, executemany() for 10-100x faster performance
    
    Args:
        db: Database connection
        table_name: Name of the table
        records: List of dictionaries with column:value pairs
        conflict_cols: List of columns to check for conflicts
        update_cols: Optional list of columns to update. If None, updates all non-conflict columns
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
    
    Example:
        bulk_upsert(db, "transactions", [{"id": 1, "amount": 150}, ...], ["id"], ["amount"])
    """
    if not records:
        return
    
    db_type = get_db_type()
    columns = list(records[0].keys())
    placeholders = [f":{col}" for col in columns]
    
    # Determine which columns to update
    if update_cols is None:
        update_cols = [col for col in columns if col not in conflict_cols]
    
    if db_type == "POSTGRESQL":
        update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_cols])
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            ON CONFLICT ({', '.join(conflict_cols)}) 
            DO UPDATE SET {update_set}
        """
    else:  # SQLite
        sql = f"""
            INSERT OR REPLACE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        # Use SQLAlchemy's execute with list for optimized bulk operation
        for record in records:
            db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to bulk upsert into {table_name}: {str(e)}") from e


In [ ]:
show_doc(bulk_upsert)

In [ ]:
show_doc(upsert)

## CRUD Operations

Standard Create, Read, Update, Delete operations.

In [ ]:
#| export

def get_by_id(db: Database, table_name: str, id_value: Any, id_col: str = "id") -> Any:
    """
    Get a single record by ID
    
    Args:
        db: Database connection
        table_name: Name of the table
        id_value: Value of the ID to search for
        id_col: Name of the ID column (default: "id")
    
    Returns:
        Query result
    
    Example:
        result = get_by_id(db, "transactions", 123, "transaction_id")
    """
    sql = f"SELECT * FROM {table_name} WHERE {id_col} = :id_value"
    try:
        result = db.conn.execute(text(sql), {"id_value": id_value})
        return result
    except Exception as e:
        raise Exception(f"Failed to get record from {table_name}: {str(e)}") from e

def update_record(db: Database, table_name: str, id_value: Any, 
                  id_col: str = "id", auto_commit: bool = True, **updates) -> None:
    """
    Update a single record by ID
    
    Args:
        db: Database connection
        table_name: Name of the table
        id_value: Value of the ID to update
        id_col: Name of the ID column (default: "id")
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
        **updates: Column=value pairs to update
    
    Example:
        update_record(db, "transactions", 123, "transaction_id", amount=200, status="completed")
    """
    if not updates:
        return
    
    set_clause = ', '.join([f"{col} = :{col}" for col in updates.keys()])
    sql = f"UPDATE {table_name} SET {set_clause} WHERE {id_col} = :id_value"
    
    params = {**updates, "id_value": id_value}
    
    try:
        db.conn.execute(text(sql), params)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to update record in {table_name}: {str(e)}") from e

def delete_record(db: Database, table_name: str, id_value: Any, id_col: str = "id", auto_commit: bool = True) -> None:
    """
    Delete a single record by ID
    
    Args:
        db: Database connection
        table_name: Name of the table
        id_value: Value of the ID to delete
        id_col: Name of the ID column (default: "id")
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
    
    Example:
        delete_record(db, "transactions", 123, "transaction_id")
    """
    sql = f"DELETE FROM {table_name} WHERE {id_col} = :id_value"
    
    try:
        db.conn.execute(text(sql), {"id_value": id_value})
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to delete record from {table_name}: {str(e)}") from e

def bulk_delete(db: Database, table_name: str, id_list: List[Any], id_col: str = "id", auto_commit: bool = True) -> None:
    """
    Delete multiple records by ID list
    
    Args:
        db: Database connection
        table_name: Name of the table
        id_list: List of ID values to delete
        id_col: Name of the ID column (default: "id")
        auto_commit: If True, commits immediately. If False, caller must commit (default: True)
    
    Example:
        bulk_delete(db, "transactions", [1, 2, 3, 4, 5], "transaction_id")
    """
    if not id_list:
        return
    
    # Create placeholder for IN clause
    placeholders = ', '.join([f":id_{i}" for i in range(len(id_list))])
    sql = f"DELETE FROM {table_name} WHERE {id_col} IN ({placeholders})"
    
    # Build params dict
    params = {f"id_{i}": val for i, val in enumerate(id_list)}
    
    try:
        db.conn.execute(text(sql), params)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to bulk delete from {table_name}: {str(e)}") from e


In [ ]:
show_doc(bulk_delete)

In [ ]:
show_doc(delete_record)

In [ ]:
show_doc(update_record)

In [ ]:
show_doc(get_by_id)

## Helper Utilities

Transaction management, pagination, and other utilities.

In [ ]:
#| export

@contextmanager
def with_transaction(db: Database):
    """
    Context manager for safe transaction handling with auto-rollback on error
    
    IMPORTANT: When using this context manager, set auto_commit=False on all
    database operations inside the block to allow proper rollback behavior.
    
    Example:
        with with_transaction(db):
            insert_only(db, "table1", {...}, ["id"], auto_commit=False)
            update_record(db, "table2", 123, auto_commit=False, amount=200)
            # Automatically commits if no errors, rolls back on exception
    """
    try:
        yield db
        db.conn.commit()
    except Exception as e:
        db.conn.rollback()
        raise e

def paginate_sql(sql: str, page: int, page_size: int) -> str:
    """
    Add pagination to a SQL query
    
    Args:
        sql: Base SQL query
        page: Page number (1-indexed)
        page_size: Number of records per page
    
    Returns:
        SQL with LIMIT and OFFSET added
    
    Example:
        paginated = paginate_sql("SELECT * FROM transactions", 2, 50)
        # Returns: "SELECT * FROM transactions LIMIT 50 OFFSET 50"
    """
    offset = (page - 1) * page_size
    return f"{sql.rstrip(';')} LIMIT {page_size} OFFSET {offset}"

def batch_execute(db: Database, operation_func, items: List[Any], batch_size: int = 100) -> None:
    """
    Execute an operation on items in batches with commits after each batch
    
    Args:
        db: Database connection
        operation_func: Function to call for each item (takes db and item as args)
        items: List of items to process
        batch_size: Number of items per batch (default: 100)
    
    Example:
        def insert_item(db, item):
            insert_only(db, "transactions", item, ["id"])
        
        batch_execute(db, insert_item, large_list_of_records, batch_size=50)
    """
    for i in range(0, len(items), batch_size):
        batch = items[i:i + batch_size]
        try:
            for item in batch:
                operation_func(db, item)
            db.conn.commit()
        except Exception as e:
            db.conn.rollback()
            raise Exception(f"Batch execution failed at batch {i//batch_size + 1}: {str(e)}") from e


In [ ]:
show_doc(batch_execute)

In [ ]:
show_doc(paginate_sql)

In [ ]:
show_doc(with_transaction)